<a href="https://colab.research.google.com/github/sanketmohanty09/CovidDetection/blob/main/CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!nvidia-smi

Sun Jun 27 08:00:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   78C    P0    35W /  70W |   9054MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
train_path='/content/drive/MyDrive/CovidDetection /Train'
test_path='/content/drive/MyDrive/CovidDetection /Val'

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

In [18]:
### CNN Based Model
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',metrics=['accuracy'],loss=keras.losses.binary_crossentropy)



In [19]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 54, 54, 64)       

In [ ]:
train_datagen=image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_dataset = image.ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator=train_datagen.flow_from_directory(
    '/content/drive/MyDrive/CovidDetection /Train',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

Found 344 images belonging to 2 classes.


In [ ]:
train_generator.class_indices

{'Covid': 0, 'NORMAL': 1}

In [ ]:
validation_generator = test_dataset.flow_from_directory(
    '/content/drive/MyDrive/CovidDetection /Val',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
    
)

Found 64 images belonging to 2 classes.


In [ ]:
hist=model.fit_generator(
    train_generator,
    steps_per_epoch=8,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=2
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
8/8 [==============================] - 144s 12s/step - loss: 1.1519 - accuracy: 0.5525 - val_loss: 0.6866 - val_accuracy: 0.8125
Epoch 2/10
8/8 [==============================] - 28s 3s/step - loss: 0.6503 - accuracy: 0.6790 - val_loss: 0.6671 - val_accuracy: 0.5625
Epoch 3/10
8/8 [==============================] - 12s 2s/step - loss: 0.5316 - accuracy: 0.7641 - val_loss: 0.5608 - val_accuracy: 0.8125
Epoch 4/10
8/8 [==============================] - 10s 1s/step - loss: 0.3604 - accuracy: 0.8769 - val_loss: 0.5690 - val_accuracy: 0.8125
Epoch 5/10
8/8 [==============================] - 9s 1s/step - loss: 0.2314 - accuracy: 0.9150 - val_loss: 0.5646 - val_accuracy: 0.8125
Epoch 6/10
8/8 [==============================] - 9s 1s/step - loss: 0.4535 - accuracy: 0.8484 - val_loss: 0.5688 - val_accuracy: 0.7812
Epoch 7/10
8/8 [==============================] - 9s 1s/step - loss: 0.2496 - accuracy: 0.9394 - val_loss: 0.7788 - val_accuracy: 0.8125
Epoch 8/10
8/8 [===================

In [ ]:
model.save('weights.h5')

In [ ]:
del model

In [20]:
model.load_weights('weights.h5')

### *Custom Image Testing*

In this part we can upload any xray image from our local drive and we will try use to use  the pre-trained model weights to predict wether the uploaded xray is covid +ve or covid negative  

In [25]:
import numpy as np

from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path='/content/' + fn
  img=image.load_img(path, target_size=(224, 224))
  
  x=image.img_to_array(img)
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])
  
  classes = model.predict(images, batch_size=10)
  
  print(classes[0])
  
  if classes[0]>0:
    print(fn + " covid -ve")
    
  else:
    print(fn + " covid +ve")

Saving NORMAL2-IM-0857-0001.jpeg to NORMAL2-IM-0857-0001.jpeg
Saving NORMAL2-IM-0860-0001.jpeg to NORMAL2-IM-0860-0001.jpeg
Saving NORMAL2-IM-0863-0001.jpeg to NORMAL2-IM-0863-0001.jpeg
Saving NORMAL2-IM-0866-0001.jpeg to NORMAL2-IM-0866-0001.jpeg
Saving NORMAL2-IM-0876-0001.jpeg to NORMAL2-IM-0876-0001.jpeg
Saving NORMAL2-IM-0886-0001.jpeg to NORMAL2-IM-0886-0001.jpeg
Saving NORMAL2-IM-0892-0001.jpeg to NORMAL2-IM-0892-0001.jpeg
Saving NORMAL2-IM-0900-0001.jpeg to NORMAL2-IM-0900-0001.jpeg
[1.]
NORMAL2-IM-0857-0001.jpeg covid -ve
[1.]
NORMAL2-IM-0860-0001.jpeg covid -ve
[1.]
NORMAL2-IM-0863-0001.jpeg covid -ve
[5.6566383e-38]
NORMAL2-IM-0866-0001.jpeg covid -ve
[1.]
NORMAL2-IM-0876-0001.jpeg covid -ve
[1.]
NORMAL2-IM-0886-0001.jpeg covid -ve
[1.]
NORMAL2-IM-0892-0001.jpeg covid -ve
[1.]
NORMAL2-IM-0900-0001.jpeg covid -ve
